<a href="https://www.kaggle.com/code/mikedelong/essay-keywords-first-look?scriptVersionId=153082917" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%env TOKENIZERS_PARALLELISM=true
! pip install sentence-transformers 

env: TOKENIZERS_PARALLELISM=true
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fb2ffd1a536bb7ac93429f7b5b4dc14bfdcdfbbcaddedf4e6625cce3a3e7c919
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
with open(file='/kaggle/input/paul-graham-essays/paul_graham_essays.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
essays = text.split('\n')
print('essay count: {}'.format(len(essays)))

essay count: 55


In [3]:
# https://www.kaggle.com/code/ianalyticsgeek/keywords-extraction-using-bert
from arrow import now
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# we can't use more features than we can visualize
MAX_FEATURES = 600

NGRAM_RANGE = (1, 1)
model_start = now()
count = TfidfVectorizer(ngram_range=NGRAM_RANGE, stop_words='english', max_features=MAX_FEATURES).fit(essays)
features = count.get_feature_names_out()
print('{}: got features'.format(now()))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# we need to encode the essays to get the words' relationships to each other
embedding = model.encode(essays)
print('{}: got embeddings'.format(now()))
feature_embeddings = model.encode(features)
print('{}: got feature embeddings'.format(now()))
print('model time: {}'.format(now() - model_start))

2023-11-30T21:36:05.730423+00:00: got features


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2023-11-30T21:36:37.524255+00:00: got embeddings


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-11-30T21:36:39.394435+00:00: got feature embeddings
model time: 0:00:34.260998


In [4]:
# post process the features 
keep = set(features)
# filter out numeric values
keep = {item for item in keep if not item.isnumeric()}
# do some basic plural identification and remove the plurals 
plurals = {item for item in keep if (item.endswith('s') and item[:-1] in keep) or (item.endswith('es') and item[:-2] in keep)}
keep = keep.difference(plurals)
print(len(keep), keep)

534 {'left', 'guess', 'world', 'effort', 'read', 'dollars', 'principle', 'companies', 'likely', 'getting', 'expect', 'possible', 'market', 'wealth', 'space', 'time', 'microsoft', 'realized', 'recently', 'thought', 'working', 'case', 'matter', 'cause', 'trying', 'hacker', 'instead', 'initially', 'money', 'common', 'variation', 'stop', 'cost', 'investor', 'blackwell', 'bad', 'hacking', 'stock', 'dangerous', 'government', 'specific', 'pay', 'mean', 'search', 'past', 'nerds', 'office', 'remember', 'precisely', 'late', 'given', 'face', 'online', 'rate', 'livingston', 'series', 'went', 'program', 'venture', 'kid', 'apple', 'essay', 'individual', 'suspect', 'existing', 'yes', 'situation', 'vc', 'consider', 'google', 'happened', 'think', 'difference', 'spent', 'problem', 'designed', 'day', 'course', 'future', 'rule', 'interesting', 'research', 'expensive', 'doing', 'ability', 'control', 'far', 'founder', 'field', 'spend', 'simple', 'opinions', 'friend', 'wrote', 'combinator', 'answer', 'techni

In [5]:
from nltk import pos_tag
from nltk import word_tokenize

noun_start = now()
keep_nouns = {word for item in essays for (word, part_of_speech) in pos_tag(word_tokenize(item)) if part_of_speech[:2] == 'NN' and word in keep}
print('filtered nouns in {}'.format(now() - noun_start))
keep = keep_nouns
print(len(keep), keep)

filtered nouns in 0:00:40.020594
348 {'left', 'guess', 'world', 'effort', 'read', 'dollars', 'principle', 'companies', 'market', 'wealth', 'space', 'time', 'thought', 'working', 'matter', 'case', 'cause', 'hacker', 'money', 'variation', 'stop', 'cost', 'investor', 'hacking', 'stock', 'government', 'specific', 'pay', 'mean', 'search', 'past', 'nerds', 'office', 'remember', 'online', 'face', 'rate', 'series', 'program', 'venture', 'kid', 'apple', 'essay', 'individual', 'suspect', 'yes', 'situation', 'think', 'difference', 'spent', 'problem', 'day', 'course', 'rule', 'future', 'interesting', 'research', 'ability', 'control', 'founder', 'field', 'simple', 'opinions', 'friend', 'answer', 'word', 'use', 'attention', 'experience', 'audience', 'drafts', 'default', 'invest', 'version', 'email', 'reading', 'bit', 'work', 'organizations', 'ones', 'application', 'paul', 'students', 'valuation', 'design', 'investment', 'asking', 'goal', 'writing', 'turn', 'valley', 'patent', 'lose', 'mind', 'self',

In [6]:
# now we can project the feature vectors into 2-space to visualize 
from pandas import DataFrame
from plotly.express import scatter
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=2023, verbose=1, n_iter=10000)
tsne_df = DataFrame(data=tsne.fit_transform(X=feature_embeddings), columns=['t0', 't1'])
tsne_df['word'] = features
scatter(data_frame=tsne_df[tsne_df['word'].isin(keep)], x='t0', y='t1', text='word', height=900, ).update_traces(marker={'size': 1})

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 600 samples in 0.001s...
[t-SNE] Computed neighbors for 600 samples in 0.183s...
[t-SNE] Computed conditional probabilities for sample 600 / 600
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 92.646370
[t-SNE] KL divergence after 1250 iterations: 1.989587
